## Assignment 5: Advance pandas
Let's continue to use the dataset from the wine magazine used in notebook 'Colab_Lec06_advanced-pandas' to practice data transformation, grouping, and sorting.

<img src="https://secure.static.meredith.com/crt/store/covers/magazines/nmo/9826_l.jpg">

In [1]:
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
import pandas as pd
wine = pd.read_csv(csvurl)
wine.drop(wine.columns[0], axis="columns", inplace=True)
wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [8]:
wine.info()
null_count = wine.isnull().sum()
print(null_count)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65499 entries, 0 to 65498
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                65467 non-null  object 
 1   description            65499 non-null  object 
 2   designation            46588 non-null  object 
 3   points                 65499 non-null  int64  
 4   price                  60829 non-null  float64
 5   province               65467 non-null  object 
 6   region_1               54744 non-null  object 
 7   region_2               25170 non-null  object 
 8   taster_name            51856 non-null  object 
 9   taster_twitter_handle  49467 non-null  object 
 10  title                  65499 non-null  object 
 11  variety                65499 non-null  object 
 12  winery                 65499 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 6.5+ MB
country                     32
description      

#### Question 1: ####  
What is the mean of the points column?

In [3]:
mean_points = wine['points'].mean()
print(mean_points)


88.43403716087269


#### Question 2: ####  
How many countries are present in this dataset? (Only count each country once)

In [4]:
country_count = wine['country'].nunique()
print(country_count)

41


#### Question 3: ####
How many times does each country appeared in this dataset? Show each country and the corresponding count (show counts in ascending order)

In [5]:
country_counts = wine['country'].value_counts().sort_index()
print(country_counts)

country
Argentina                  1907
Armenia                       1
Australia                  1177
Austria                    1635
Bosnia and Herzegovina        1
Brazil                       31
Bulgaria                     68
Canada                      108
Chile                      2258
Croatia                      44
Cyprus                        6
Czech Republic                6
England                      36
France                    11174
Georgia                      37
Germany                    1051
Greece                      242
Hungary                      61
India                         4
Israel                      259
Italy                     10005
Lebanon                      20
Luxembourg                    4
Macedonia                     6
Mexico                       31
Moldova                      30
Morocco                      11
New Zealand                 733
Peru                          8
Portugal                   2963
Romania                      67


#### Question 4: ####
Create a variable `adjusted_price` containing the adjusted price which is the price subtracted by the average price. *This is called **"centering" transformation** - a method commonly used in the preprocessing step before applying various machine learning algorithms.*

In [6]:
adjusted_price = wine['price'] - wine['price'].mean()
print(adjusted_price)

0              NaN
1       -20.232932
2       -21.232932
3       -22.232932
4        29.767068
           ...    
65494     9.767068
65495   -13.232932
65496   -15.232932
65497    -4.232932
65498   -25.232932
Name: price, Length: 65499, dtype: float64


#### Question 5: ####
What is the title of the wine that has the highest points-to-price ratio in the dataset?

In [9]:
points_to_price_ratio = wine['points'] / wine['price']
max_ratio_index = points_to_price_ratio.idxmax()
wine.loc[max_ratio_index, 'title']

'Bandit NV Merlot (California)'

#### Question 6: ####
Create a series `flavor_counts` that contains two values: the number of wines that has the word "tart" in the `description` column and the number of wines that has the word "berries" in the `description` column. The index of the Series should be "Tart" and "Berries" for the corresponding values.

In [11]:
# Look for the word "tart", not case sensitive, not a substring of another word.
# Same applies to the word "berries"
# Your code here . . .

flavor_counts = pd.Series(
    [wine['description'].str.contains('tart', case=False).sum(),
     wine['description'].str.contains('berries', case=False).sum()],
    index=['Tart', 'Berries']
)

print(flavor_counts)

Tart       4424
Berries    3512
dtype: int64


#### Question 7: ####
Let's convert the points into simple star ratings. A score of 90 or higher counts as 3 stars, a score of at least 80 but less than 90 is 2 stars. Any other score is 1 star.

Also, any wines from France should automatically get 3 stars, regardless of points.

Add this new column `star_ratings` to the dataframe with the number of stars for each wine in the dataset.

In [12]:
star_ratings = wine.points.apply(lambda x: 3 if x >= 90 else (2 if x >= 80 else 1))
star_ratings[wine.country == "France"] = 3
wine["star_ratings"] = star_ratings
wine.head(20)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,star_ratings
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2
7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,3
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,2
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,3


#### Question 8: ####
Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.

In [16]:
taste_twitter_handle_counts = wine.groupby("taster_twitter_handle").taster_twitter_handle.value_counts().sort_values(ascending=False)
taste_twitter_handle_counts

,count
taster_twitter_handle,
@vossroger,13045
@wineschach,7752
@kerinokeefe,5313
@paulgwine,4851
@vboone,4696
@mattkettmann,3035
@JoeCz,2605
@wawinereport,2358
@gordone_cellars,2032


#### Question 9: ####
What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.

In [17]:
combinations = wine.groupby(["country", "variety"]).variety.count().sort_values(ascending=False)
combinations

country  variety                 
US       Pinot Noir                  4918
         Cabernet Sauvignon          3649
         Chardonnay                  3412
France   Bordeaux-style Red Blend    2380
Italy    Red Blend                   1870
                                     ... 
         Torbato                        1
         Vespaiolo                      1
         Vespolina                      1
         Vitovska                       1
Uruguay  Tempranillo-Tannat             1
Name: variety, Length: 1304, dtype: int64

#### Question 10 #####
Create a Series whose index is reviewers and whose values is the average score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [18]:
average_scores = wine.groupby("taster_name").points.mean()
average_scores


,points
taster_name,
Alexander Peartree,86.014286
Anna Lee C. Iijima,88.380506
Anne Krebiehl MW,90.587903
Carrie Dykes,86.644444
Christina Pickard,89.500000
Fiona Adams,87.090909
Jeff Jenssen,88.273504
Jim Gordon,88.604331
Joe Czerwinski,88.519770
